In [11]:
import numpy as np
import rdkit as rd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity

In [12]:
#Determines if SMILES is valid or not
def is_valid(smiles):
    
    mol = Chem.MolFromSmiles(smiles)

    #Returns True if SMILES is valid, returns False if SMILES is invalid
    return smiles != '' and mol is not None and mol.GetNumAtoms() > 0

In [13]:
#Determines LogP
def logP(smiles):
    
    return(Descriptors.MolLogP(Chem.MolFromSmiles(smiles)))

In [14]:
#Determins molecular weight
def molWt(smiles):
    
    return(Descriptors.MolWt(Chem.MolFromSmiles(smiles)))

In [15]:
#Determine number hydrogen bond acceptors
def numAcc(smiles):
    
    return(Descriptors.NumHAcceptors(Chem.MolFromSmiles(smiles)))

In [16]:
#Determine number hydrogen bond donors
def numDon(smiles):
    
    return(Descriptors.NumHDonors(Chem.MolFromSmiles(smiles)))

In [17]:
#Determine polar surface area
def polSur(smiles):
    
    return(Descriptors.TPSA(Chem.MolFromSmiles(smiles)))

In [18]:
#Determine number of rotatable bonds
def rolBon(smiles):
    return(Descriptors.NumRotatableBonds(Chem.MolFromSmiles(smiles)))

In [19]:
#Number of characters generated
num_characters = 0

#Number of molecules generated
num_smiles = 0

#Number of unique SMILES
num_unq_mols = 0

#Number of unique SMILES that aren't in the training data
num_mols = 0

#Number of valid molecules that aren't in the training data generated
num_valid = 0

#List of smiles in file, to make sure smiles are unique
smileslist = []

In [20]:
#Test how many molecules are valid (without considering uniqueness, novelty)
total = 0
number_valid = 0

#Read in data file line by line
for line in open("generatedsmiles.txt", "r"):
    total += 1
    
    #Ensure smiles are valid
    if(is_valid(line) == True):
                           
        #Increment number of valid molecules generated
        number_valid += 1

print("Percent valid molecules: " + str(number_valid / total * 100))

Percent valid molecules: 25.10394502698961


In [21]:
#Training data
training_data = list(open("smiles.txt", "r"))

#File with unique generated SMILES that aren't in the training data
generatedmols = open("generatedmols.txt", "w")

#Read in data file line by line
for line in open("generatedsmiles.txt", "r"):
    
    #Ensure molecules are unique
    if line not in smileslist:
        
        smileslist.append(line)
        
        num_unq_mols += 1

        #Ensure smiles aren't in training data
        if line not in training_data:  

            #Remove \n character, remove G character
            smiles = line.replace("\n", "").replace("G", "")
            
            #Increment number of molecules generated
            num_mols += 1

            #Ensure smiles are valid
            if(is_valid(smiles) == True):
            
                #Copy over SMILES satisfying requirements
                generatedmols.write(smiles + "\n")
                
                #Increment number of valid molecules generated
                num_valid += 1
                
    #Increment total number of SMILES generated
    num_smiles += 1
    
    #Add length of line to total number of characters
    num_characters += len(line)

In [22]:
print("Number of characters generated: " + str(num_characters))
print("Number of molecules generated: " + str(num_smiles))
print("Number of unique molecules generated: " + str(num_unq_mols))
print("Number of novel and unique molecules generated: " + str(num_mols))
print("Number of novel, unique, and valid molecules generated: " + str(num_valid))

Number of characters generated: 9791155
Number of molecules generated: 235461
Number of unique molecules generated: 178821
Number of novel and unique molecules generated: 178821
Number of novel, unique, and valid molecules generated: 13308


In [23]:
#List of Morgan fingerprints of molecules
fingerprints = []

#Read in data file line by line
for line in open("generatedmols.txt", "r"):
    
    #Convert SMILES string to Morgan fingerprint
    mol = Chem.MolFromSmiles(line.replace("\n", ""))
    fingerprint = AllChem.GetMorganFingerprint(mol, 2)
    
    #Add to list of fingerprints
    fingerprints.append(fingerprint)    

In [24]:
#Total Tanimoto Distance
tanimoto = 0

#Calculate Tanimoto Distance between each pair of fingerprints
for fpt1 in fingerprints:
    for fpt2 in fingerprints:
        
        if fpt1 != fpt2:
            
            #Calculate Tanimoto Distance
            tan = TanimotoSimilarity(fpt1, fpt2)
            tanimoto += tan

#Average Tanimoto Distance (internal diversity)
avg_tanimoto = (1 / (num_valid ** 2)) * tanimoto
print("Average Internal Diversity: {:0.4f}".format(avg_tanimoto))

Average Internal Diversity: 0.1256


In [ ]:
#Array of molecular properties for generated molecules
molProps = np.empty((0,5))

#Read in data file line by line
for molecule in open("generatedmols.txt", "r"):
    try:
        #Array of properties [partition coefficient, molecular weight, number of hydrogen bond acceptors, number of hydrogen bond donors, polar surface area]
        props = np.reshape(np.array([logP(molecule), molWt(molecule), numAcc(molecule), numDon(molecule), rolBon(molecule)]), (1, 5))
    
        #Append properties
        molProps = np.append(molProps, props, axis=0)
        
    except:
        #Occasionally RDKit bugs don't allow for analyzing the molecule; in these cases, simply remove the molecule
        molecule.replace(molecule, "")
        continue

In [ ]:
#Array of number of molecules each molecule is dominated by
dom = np.zeros((np.shape(molProps)[0]))

#Analyze each molecule's properties as they compare to others
for i in range(np.shape(molProps)[0]):
    
    for j in range(np.shape(molProps)[0]):
        
        #Compare each property between the molecules
        if (molProps[j, 0] <= molProps[i, 0]):
            
            if (molProps[j, 1] <= molProps[i, 1]):
            
                if (molProps[j, 2] <= molProps[i, 2]):

                    if (molProps[j, 3] <= molProps[i, 3]):
            
                        if (molProps[j, 4] <= molProps[i, 4]):
                            
                            #If molecule j is better than molecule i in every property, than j dominates i
                            dom[i] += 1

In [ ]:
#Count how many frontiers there are and how many molecules are in each frontier
unique, counts = np.unique(dom, return_counts=True)
fronts = dict(zip(unique, counts))

#Indices of best molecules for use in finetuning
fineMols = np.empty((0,1))

#Fraction of molecules to be used for transfer learning
top = 0.5

#Frontier iterator 
f = 0
#Get indices of the best (top) fraction of molecules
while(np.shape(fineMols)[0] < (top * np.shape(molProps)[0])):

    #Indices of molecules in frontier i
    fineMolecules = np.transpose(np.where(dom == f))
    
    #Add to finemols
    fineMols = np.append(fineMols, fineMolecules, axis=0)
      
    #Look at next worst frontier
    f += 1

In [ ]:
#Save best molecules to transferdata.txt
transferdata = open("transferdata.txt", "w")

#Index counter
i = 0

#Get best molecules and save them to transferdata.txt
for line in open("generatedmols.txt", "r"):
    
    #Get best molecules, write to new file
    if i in fineMols:
        
        #Append start token
        line = line.rjust(len(line)+1, "G")
        
        #Write to file
        transferdata.write(line)
    
    i += 1

#Close file
transferdata.close()

In [ ]:
#Array of molecular properties for generated molecules
transferMolProps = np.empty((0,5))

#Read in data file line by line
for molecule in open("transferdata.txt", "r"):
    
    #Remove start token to analyze molecules
    molecule = molecule.replace("G", "")
    
    #Array of properties [partition coefficient, molecular weight, number of hydrogen bond acceptors, number of hydrogen bond donors, polar surface area]
    transferProp = np.reshape(np.array([logP(molecule), molWt(molecule), numAcc(molecule), numDon(molecule), rolBon(molecule)]), (1, 5))
    
    #Append properties
    transferMolProps = np.append(transferMolProps, transferProp, axis=0)

In [ ]:
#Read in original training data file to get character list
data = open("smiles.txt", "r").read()

#Create a list of the unique characters in the dataset
chars = list(set(data))

In [ ]:
from numpy import argmax
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Create array from characters in the dataset
values = array(chars)
valueslist = values.tolist()
print("Array of unique characters:")
print(values)

#Create unique, numerical labels for each character between 0 and n-1, where n is the number of unique characters
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print("Array of labels for each character:")
print(integer_encoded)

#Encode characters into a one-hot encoding, resulting in an array of size [num unique chars, num unique chars]
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print("Array of one-hot encoded characters:")
print(onehot_encoded)
print("Size of array of one-hot encoded characters: " + str(onehot_encoded.shape))

In [ ]:
#Read in transfer data file
data = open("transferdata.txt", "r").read()
#Create a list of the dataset, along with all the characters to ensure they are all represeented
datalist = list(data)
datalist.extend(valueslist)
#Create an array of the dataset
dataarray = array(datalist)
#Fit one-hot encoding to dataarray
dataarray = dataarray.reshape(len(dataarray), 1)
#Fit encoder, remove all characters at the end leaving just the molecules
ohefinemols = onehot_encoder.fit_transform(dataarray).astype(int)[0:31980,:]
print("Size of one-hot encoded array of data: " + str(ohefinemols.shape))
print("One-hot encoded array of data:")
print(ohefinemols)

In [ ]:
#Save ohefinemols as a (compressed) file
np.savez_compressed("ohefinemols.npz", ohefinemols)

In [ ]:
#Create integer transfer data
intfinemols = [np.where(r==1)[0][0] for r in ohefinemols]

In [ ]:
#Save intfinemols as a (compressed) file
np.savez_compressed("intfinemols.npz", intfinemols)

In [ ]:
#Array with SMILES character, integer encoding, and one hot encoding (vocabulary)
values = np.reshape(values, (np.shape(values)[0], 1))
vocab = np.concatenate((values, integer_encoded.astype(object)), axis = 1)
vocab = vocab[vocab[:,1].argsort()]
vocabvalues = np.reshape(vocab[:,1], (-1,1))
vocabohe = onehot_encoder.fit_transform(vocabvalues)
vocabencodings = np.concatenate((vocab, vocabohe.astype(object)), axis = 1)
print(np.shape(vocabencodings))

In [ ]:
print(vocabencodings)

In [ ]:
#PCA visualization
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

#Scale array of properties for all molecules
scaledProps = StandardScaler().fit_transform(np.concatenate((molProps, transferMolProps), axis=0))

#Apply PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(scaledProps)

In [ ]:
#Graph
import matplotlib.pyplot as plt

plt.scatter(principalComponents[0:np.shape(molProps)[0],0],principalComponents[0:np.shape(molProps)[0],1], 75)
plt.scatter(principalComponents[np.shape(molProps)[0]:,0],principalComponents[np.shape(molProps)[0]:,1], 25)
plt.show()